# Database Setup with DuckDB

This notebook sets up a local DuckDB database with three tables:
- **users**: User information
- **transactions**: Financial transactions
- **activity**: User activity records

## ERD Overview
- `users` has one-to-many relationship with `transactions`
- `users` has one-to-many relationship with `activity`
- `transactions` has one-to-many relationship with `activity`


In [1]:
import duckdb
import pandas as pd
import uuid
from datetime import datetime, timedelta
import random

# Create or connect to DuckDB database
conn = duckdb.connect('trial_db.duckdb')
print("Connected to DuckDB database!")


Connected to DuckDB database!


## Create Tables

Creating the three tables based on the ERD schema.


In [56]:
# Drop tables if they exist (for clean setup)
conn.execute("DROP TABLE IF EXISTS activity")
conn.execute("DROP TABLE IF EXISTS transactions")
conn.execute("DROP TABLE IF EXISTS users")

# Create users table
conn.execute("""
    CREATE TABLE users (
        user_id UUID PRIMARY KEY,
        created_date TIMESTAMP,
        country TEXT,
        plan TEXT
    )
""")

# Create transactions table
conn.execute("""
    CREATE TABLE transactions (
        transaction_id UUID PRIMARY KEY,
        user_id UUID,
        ip_address TEXT,
        created_date TIMESTAMP,
        direction TEXT,
        amount_gbp DOUBLE,
        state TEXT,
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    )
""")

# Create activity table
conn.execute("""
    CREATE TABLE activity (
        activity_id UUID PRIMARY KEY,
        session_id UUID,
        user_id UUID,
        created_date TIMESTAMP,
        product TEXT,
        type TEXT,
        transaction_id UUID,
        FOREIGN KEY (user_id) REFERENCES users(user_id),
        FOREIGN KEY (transaction_id) REFERENCES transactions(transaction_id)
    )
""")

print("Tables created successfully!")


Tables created successfully!


## Insert Mock Data

Generating and inserting mock data for all three tables.


In [57]:
# Generate mock users
countries = ['UK', 'US', 'CA', 'AU', 'DE', 'FR', 'ES', 'IT']
plans = ['basic', 'premium', 'enterprise', 'free']

users_data = []
num_users = 5000
base_date = datetime.now() - timedelta(days=365)

for i in range(num_users):
    users_data.append({
        'user_id': str(uuid.uuid4()),
        'created_date': base_date + timedelta(days=random.randint(0, 365)),
        'country': random.choice(countries),
        'plan': random.choice(plans)
    })

users_df = pd.DataFrame(users_data)
conn.execute("INSERT INTO users SELECT * FROM users_df")
print(f"Inserted {len(users_data)} users")


Inserted 5000 users


In [58]:
# Generate mock transactions
directions = ['incoming', 'outgoing']
states = ['pending', 'completed', 'failed']

transactions_data = []
num_transactions = 20000

# Get all user IDs
user_ids = users_df['user_id'].tolist()

for i in range(num_transactions):
    user_id = random.choice(user_ids)
    created_date = base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))
    
    transactions_data.append({
        'transaction_id': str(uuid.uuid4()),
        'user_id': user_id,
        'ip_address': f"{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}.{random.randint(1, 255)}",
        'created_date': created_date,
        'direction': random.choice(directions),
        'amount_gbp': round(random.uniform(10.0, 5000.0), 2),
        'state': random.choice(states)
    })

transactions_df = pd.DataFrame(transactions_data)
conn.execute("INSERT INTO transactions SELECT * FROM transactions_df")
print(f"Inserted {len(transactions_data)} transactions")


Inserted 20000 transactions


In [62]:
# Generate mock activity
products = ['crypto', 'trading', 'savings', 'cards', 'insurance', 'investments', 'loans']
activity_types = ['VIEW', 'CLICK', 'TRANSFER', 'DEPOSIT', 'WITHDRAWAL', 'PAYMENT', 'PURCHASE', 'LOGIN', 'LOGOUT']

activity_data = []
num_activities = 50000

# Get transaction IDs (some activities may not have transactions)
transaction_ids = transactions_df['transaction_id'].tolist()

for i in range(num_activities):
    user_id = random.choice(user_ids)
    created_date = base_date + timedelta(days=random.randint(0, 365), hours=random.randint(0, 23))
    
    # Some activities are linked to transactions, some are not (30% have no transaction)
    transaction_id = random.choice(transaction_ids) if random.random() > 0.3 else None
    
    activity_data.append({
        'activity_id': str(uuid.uuid4()),
        'session_id': str(uuid.uuid4()),
        'user_id': user_id,
        'created_date': created_date,
        'product': random.choice(products),
        'type': random.choice(activity_types),
        'transaction_id': transaction_id
    })

activity_df = pd.DataFrame(activity_data)
# Use pd.NA for NULL values (DuckDB will handle this correctly)
activity_df['transaction_id'] = activity_df['transaction_id'].where(activity_df['transaction_id'].notna(), pd.NA)
conn.execute("INSERT INTO activity SELECT * FROM activity_df")
print(f"Inserted {len(activity_data)} activities")


Inserted 50000 activities


## Verify Data

Let's check the data we've inserted.


In [63]:
# Check table counts
print("Table row counts:")
print(f"Users: {conn.execute('SELECT COUNT(*) FROM users').fetchone()[0]}")
print(f"Transactions: {conn.execute('SELECT COUNT(*) FROM transactions').fetchone()[0]}")
print(f"Activity: {conn.execute('SELECT COUNT(*) FROM activity').fetchone()[0]}")


Table row counts:
Users: 5000
Transactions: 20000
Activity: 100000


In [64]:
# Preview users table
print("\nUsers sample:")
conn.execute("SELECT * FROM users LIMIT 5").df()



Users sample:


,user_id,created_date,country,plan
0,016fe821-ffb5-4436-8ac5-7a5befed5882,2025-11-16 18:25:42.928220,IT,enterprise
1,9c3a6fc1-d8c2-406f-ba7c-731daaaeaf45,2025-10-31 18:25:42.928220,ES,basic
2,e79368ad-e456-480a-8386-e55bbac87f77,2024-12-13 18:25:42.928220,IT,enterprise
3,5d479703-599a-4892-be13-1c11d337d026,2025-02-20 18:25:42.928220,FR,basic
4,9179f3d0-e7f2-4270-bcfd-4f3a59bd5165,2025-10-09 18:25:42.928220,US,premium


In [65]:
# Preview transactions table
print("\nTransactions sample:")
conn.execute("SELECT * FROM transactions LIMIT 5").df()



Transactions sample:


,transaction_id,user_id,ip_address,created_date,direction,amount_gbp,state
0,e321bec9-13be-4e88-87c9-d684a2730455,7b089ef4-b5dc-4a25-925d-b49f63495885,51.94.134.106,2025-05-26 02:25:42.928220,outgoing,2481.92,failed
1,34579589-9d4a-4a68-98ef-3d61d64cb182,b1f38761-7187-4d19-8e1a-c715461af391,158.204.102.52,2025-10-26 08:25:42.928220,incoming,4800.80,failed
2,48c0f686-00cc-458d-aa99-eb29f7c0c8a7,0a5a3a8d-0558-476e-a33f-7003acc96f33,208.81.244.215,2025-07-23 19:25:42.928220,incoming,2171.63,failed
3,ccbe4534-8727-47eb-9f96-b45a9f5fb892,9b6de98c-4782-471d-8569-9c59b11c6de2,86.208.4.44,2025-06-01 05:25:42.928220,outgoing,927.17,completed
4,24f152b7-b2a5-480b-be2c-6217565efbb7,e9d166a3-6c38-419c-995d-c3d86874ad4d,76.117.18.153,2025-06-01 15:25:42.928220,outgoing,2441.57,completed


In [66]:
# Preview activity table
print("\nActivity sample:")
conn.execute("SELECT * FROM activity LIMIT 5").df()



Activity sample:


,activity_id,session_id,user_id,created_date,product,type,transaction_id
0,9a0067c3-e8d8-4be4-bb5a-074ff1649bc7,fcaf02c7-d347-41d9-9b74-8585270bffa8,876f1f55-1798-49c5-8a84-88ac757a9ab2,2025-01-01 15:25:42.928220,insurance,PAYMENT,5501d540-f7dd-42f3-92e3-e79ff6f71180
1,21dc85a6-c095-45ef-a4ae-f850a2053ca3,deb45bbb-13a6-4ccb-80bc-13cc36b74d30,1f241118-bb65-43c6-8472-5096aea2843b,2025-03-02 00:25:42.928220,loans,PURCHASE,8942da1c-5aa6-40a6-9b4d-fc8dcd098ba9
2,745f13ed-52b5-4c49-b385-30c5dc516425,2f9d1ca2-0132-47a3-9c4f-610acf3c1c44,f0127418-ca50-4f81-a504-5e473dbb38aa,2025-07-16 02:25:42.928220,loans,LOGIN,caf8c0c2-ad10-4bc9-8bc0-75078e516410
3,23fcb52d-4037-40fc-ab8b-558179bc73bc,f1be595e-2134-4bd0-bc8f-16cfb54eb19b,6d98600c-f2c1-44e1-a6ed-0dbc13fae843,2024-12-29 06:25:42.928220,loans,VIEW,<NA>
4,ebfd12cd-6a0a-47bb-92b6-dfd9cc185573,6d6461bc-e018-4d97-bd05-d7e1f3d99fdc,8fb49830-b9e8-4799-9383-8e73d58935af,2025-05-26 23:25:42.928220,cards,PURCHASE,035f333c-515c-4c6b-ac9c-82423dfcc4d0


In [67]:
# users whose total volume of transaction in 
# CRYPTO is greater than 100 within the first 7 
# days since singn up

query = """
    
SELECT 
    u.user_id,
    SUM(t.amount_gbp) AS total_volume
FROM users u
INNER JOIN transactions t ON t.user_id = u.user_id
    AND t.created_date BETWEEN u.created_date AND u.created_date + INTERVAL 7 DAY
INNER JOIN activity a ON t.transaction_id = a.transaction_id
    AND a.product = 'crypto'
    AND a.type = 'PURCHASE'
GROUP BY u.user_id
HAVING SUM(t.amount_gbp) > 100;

"""


conn.execute(query).df()

,user_id,total_volume
0,6a584388-ab1e-4a56-9788-b2ce144e1aff,3105.89
1,948bef83-caeb-43a4-a290-589cc710ddff,4595.05
2,62cc8b9f-1329-4c05-b360-0db7540db8e4,1395.62
3,b1ffeda7-0fd4-48e1-a08e-59ecea5ace3c,1388.31
4,2783c23e-170a-4cf7-b71c-08f0eb9ae3b7,4606.84
5,9653c669-5137-4d77-9133-ccd5c9c42f0a,3714.33
6,be0e4230-16f6-4304-a9bb-6e57a529c8d8,3830.53
7,73e1aa9d-3bfe-41ec-904b-360f62d6be91,4902.74
8,d6a732e7-8d12-473e-b4a0-33d366b6f87f,4623.98
9,cf15da18-f520-46a4-9ad5-0bf8158b4c4a,4821.38


In [68]:
# users whose total volume of transaction in 
# CRYPTO is greater than 100 within the first 7 
# days since singn up

query = """
    
-- Optimized query for large tables
WITH filtered_activity AS (
    -- Pre-filter activity table to only crypto activities
    SELECT DISTINCT transaction_id
    FROM activity
    WHERE product = 'crypto'
    AND type = 'PURCHASE'
),
user_date_ranges AS (
    -- Pre-compute date ranges for each user
    SELECT 
        user_id,
        created_date AS signup_date,
        created_date + INTERVAL 7 DAY AS end_date
    FROM users
),
filtered_transactions AS (
    -- Join transactions with filtered activity and user date ranges
    -- Apply date filter early
    SELECT 
        t.transaction_id,
        t.user_id,
        t.amount_gbp,
        udr.signup_date
    FROM transactions t
    INNER JOIN filtered_activity fa ON t.transaction_id = fa.transaction_id
    INNER JOIN user_date_ranges udr ON t.user_id = udr.user_id
    WHERE t.created_date BETWEEN udr.signup_date AND udr.end_date
)
SELECT 
    user_id,
    SUM(amount_gbp) AS total_volume
FROM filtered_transactions
GROUP BY user_id
HAVING SUM(amount_gbp) > 100;
"""


conn.execute(query).df()

,user_id,total_volume
0,bbd7bc64-6681-43d8-b280-434484ee6403,2842.46
1,cf15da18-f520-46a4-9ad5-0bf8158b4c4a,4821.38
2,f8487d33-b510-4947-8244-d8b51e735768,4089.40
3,3d166f08-9488-40a0-aa8f-38d24a97919c,3173.60
4,dcbfafec-05d1-42e2-89b0-16f474e1d902,850.62
5,5fe84088-1958-4f4c-ab61-b2695466cdec,1759.37
6,6a584388-ab1e-4a56-9788-b2ce144e1aff,3105.89
7,948bef83-caeb-43a4-a290-589cc710ddff,4595.05
8,b1ffeda7-0fd4-48e1-a08e-59ecea5ace3c,1388.31
9,2783c23e-170a-4cf7-b71c-08f0eb9ae3b7,4606.84


In [71]:
# Find which product has the highest ctr in all
# the activity table
query = """
WITH VIEW_COUNT AS (SELECT
    PRODUCT,
    COUNT(*) AS VIEW_COUNT
FROM activity
WHERE TYPE = 'VIEW'
GROUP BY PRODUCT)

, CLICK_COUNT AS (SELECT
    PRODUCT,
    COUNT(*) AS CLICK_COUNT
FROM activity
WHERE TYPE = 'CLICK'
GROUP BY PRODUCT)

SELECT
    V.PRODUCT,
    V.VIEW_COUNT,
    C.CLICK_COUNT,
    (C.CLICK_COUNT / V.VIEW_COUNT) AS CTR
FROM VIEW_COUNT V   
LEFT JOIN CLICK_COUNT C ON V.PRODUCT = C.PRODUCT
ORDER BY CTR DESC
LIMIT 10;
"""
conn.execute(query).df()


,product,VIEW_COUNT,CLICK_COUNT,CTR
0,crypto,1536,1584,1.031250
1,trading,1572,1616,1.027990
2,cards,1565,1605,1.025559
3,savings,1535,1573,1.024756
4,loans,1558,1587,1.018614
5,insurance,1573,1567,0.996186
6,investments,1564,1539,0.984015


In [ ]:
# Close connection
conn.close()
print("Database connection closed.")
